# Classifcation of MNIST data using pytorch

In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import numpy as np
from torchvision import datasets, transforms
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
transform = transforms.Compose([transforms.ToTensor()])

mnist_pytorch = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

mnist_pytorch_test = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader_pytorch = torch.utils.data.DataLoader(mnist_pytorch, batch_size=1, shuffle=False)

test_loader_pytorch = torch.utils.data.DataLoader(mnist_pytorch_test, batch_size=1, shuffle=False)



In [8]:
mnist_pytorch_train_data = mnist_pytorch.data.to(device) / 255
mnist_pytorch_train_labels = mnist_pytorch.targets.to(device)

mnist_pytorch_test_data = mnist_pytorch_test.data.to(device) / 255
mnist_pytorch_test_labels = mnist_pytorch_test.targets.to(device)


In [16]:
class ClassifcationNN(nn.Module):
    '''
    Class for a feed forward neural network to classify MNIST data 
    '''
    def __init__(self, input_size : int = 28, layer_1_size:int = 800, output_size : int = 10):
        super(ClassifcationNN, self).__init__()
        self.input_layer = nn.Linear(input_size * input_size, layer_1_size)
        self.first_layer = nn.Linear(layer_1_size, output_size)

        self.ReLU = nn.ReLU()
       

    def forward(self, input_data : np.ndarray) -> np.ndarray:
        # Forward propagation
        out = input_data.view(-1, 28*28)
        out = self.input_layer(out)
        out = self.ReLU(out)
        out = self.first_layer(out)

        return out
    
    
def train(input_data : torch.Tensor, target_data : torch.Tensor , model : nn.Module, num_epoches :int = 500) -> nn.Module:
    # Function for training a model
    criterion = nn.CrossEntropyLoss()
    optimize = optim.Adam(model.parameters(),lr = 0.001 )

    model.to(device)

    for _ in range(num_epoches):
        model.train()
        # Forward pass
        outputs = model(input_data)
        loss = criterion(outputs ,target_data)
        
        # Backward pass and optimization
        optimize.zero_grad()  # Clear gradients
        loss.backward()  # Backpropagation
        optimize.step()  # Update weights
        
    return model
   

def predict(model, input_data, real_data):
    # Set model to evaluation mode
    model.eval()

    # Ensure input data is on the correct device
    input_data = input_data.to(device)
    real_data = real_data.to(device)

    # Disable gradient calculation
    with torch.no_grad():
        # Forward pass through the model
        outputs = model(input_data)

        # Get predicted classes (index of max logit value)
        _, predicted_classes = torch.max(outputs, 1)
        
        correct = (predicted_classes == real_data).sum().item()
        # Calculate accuracy
        total = real_data.size(0)
        accuracy = 100 * correct / total

    return accuracy, predicted_classes

In [17]:
model = ClassifcationNN()

trained_model = train(mnist_pytorch_train_data, mnist_pytorch_train_labels, model )

In [18]:
predict(trained_model, mnist_pytorch_test_data, mnist_pytorch_test_labels)


(97.99, tensor([7, 2, 1,  ..., 4, 5, 6], device='cuda:0'))

: 